<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.12. Wave optics: Diffraction

Building upon our understanding of interference and polarization, which highlighted the wave nature of light, we now turn to another key wave phenomenon: **diffraction**. 
While interference involves the superposition of waves from multiple sources, diffraction describes the bending of waves as they pass through an aperture or around an obstacle. 
This bending is a direct consequence of the wave nature of light and becomes particularly noticeable when the size of the aperture or obstacle is comparable to the wavelength of the light. 


## 2.12.X Revisiting the double-slit experiment: Diffraction

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


## 2.12.X TBA

...


# Revisiting the Double-Slit Experiment and the Emergence of Diffraction

The double-slit experiment, famously known as **Young's Double-Slit Experiment**, elegantly demonstrates the wave nature of light through interference, resulting in a pattern of bright and dark fringes on a screen. Constructive interference, where path differences are integer multiples of the wavelength, leads to bright fringes, while destructive interference, with half-integer multiples, results in dark fringes. While simple interference predicts equally bright fringes, in reality, the intensity of these fringes is modulated, with the central ones being the brightest, and their intensity decreasing as you move away from the center. This suggests that each slit itself is acting as a source of waves that interfere with each other in a more complex way. This phenomenon, responsible for the intensity variations, is called **diffraction**.

Interference and diffraction are fundamentally the same phenomenon, both stemming from the superposition of coherent waves. The distinction often lies in the source arrangement: **interference** typically involves superposition from a few discrete sources, like the two slits, whereas **diffraction** involves superposition from a continuous distribution of sources or a large number of closely spaced sources. Think of water waves spreading out after passing through a narrow opening in a barrier. Light behaves similarly, bending around obstacles or spreading out after passing through narrow apertures. Essentially, diffraction can be seen as the interference of a wave with itself, where each point on a wavefront acts as a source of secondary wavelets (Huygens' principle).

# Single-Slit Diffraction and Intensity

To mathematically understand diffraction, consider monochromatic light passing through a single narrow slit of width *D*. This results in a diffraction pattern on a distant screen, characterized by a central bright maximum flanked by minima and weaker secondary maxima.

## Formation of the Diffraction Pattern

The diffraction pattern arises from the interference of waves originating from different parts of the slit.

* **Central Maximum:** Rays passing straight through the slit are in phase, creating a central bright region at an angle θ = 0.
* **Minima:** Minima occur at angles θ where the path difference between rays from the top and bottom of the slit is an integer multiple of the wavelength λ:
    $$D \sin\theta = m\lambda, \quad m = \pm 1, \pm 2, \pm 3, ...$$
* **Weak Maxima:** Between the minima, weaker maxima appear approximately where the path difference is a half-integer multiple of the wavelength:
    $$D \sin\theta \approx (m + \frac{1}{2})\lambda, \quad m = \pm \frac{3}{2}, \pm \frac{5}{2}, ...$$

## Intensity in Single-Slit Diffraction Pattern

The intensity of this diffraction pattern can be analyzed using the phasor technique. The intensity pattern can be rigorously derived by considering the superposition of the electric fields from every point within the slit. This is often done using a technique called the **phasor technique**, which essentially treats the electric field contributions from different parts of the slit as vectors and sums them up. The intensity *I<sub>u</sub>* at an angle *u* is given by:

$$I_u = I_0 \left( \frac{\sin(\beta/2)}{\beta/2} \right)^2$$

where *I<sub>0</sub>* is the intensity at the central maximum, and *β* is the total phase difference between waves from the top and bottom of the slit: The phase difference *β* arises because light waves from different points across the slit travel different distances to a point on the screen. The wave from the top edge travels a different path length than the wave from the bottom edge, leading to this phase difference. The phase difference *β* is given by:

$$\beta = \frac{2\pi}{\lambda} D \sin u$$

Substituting *β*:

$$I_u = I_0 \left( \frac{\sin\left(\frac{\pi D \sin u}{\lambda}\right)}{\frac{\pi D \sin u}{\lambda}} \right)^2$$

The function $(\frac{\sin(x)}{x})^2$, often called the sinc squared function, has a central maximum at *x* = 0 and its amplitude decreases for larger values of *x*, with zeros occurring at multiples of π. This mathematical form explains the central bright maximum and the decreasing intensity of the secondary maxima in the single-slit diffraction pattern. Minima occur when $\sin(\beta/2) = 0$, leading to $D \sin u = m\lambda$ for $m = \pm 1, \pm 2, \pm 3, ...$. Secondary maxima, with much lower intensities, appear roughly halfway between these minima.

# Diffraction in the Double-Slit Experiment (Revisited)

In reality, each slit in a double-slit experiment has a finite width *D*, and they are separated by a distance *d*. The resulting intensity pattern is a superposition of interference due to the two slits and diffraction from each individual slit. The intensity is given by:

$$I_u = I_0 \left( \frac{\sin(\beta/2)}{\beta/2} \right)^2 \cos^2\left(\frac{\delta}{2}\right)$$

where $\frac{\beta}{2} = \frac{\pi D \sin u}{\lambda}$ (diffraction factor) and $\frac{\delta}{2} = \frac{\pi d \sin u}{\lambda}$ (interference factor). The diffraction factor acts as an envelope modulating the finer interference fringes.

## Missing Orders

When a diffraction minimum coincides with an interference maximum, that maximum is absent in the observed pattern, occurring when $\frac{d}{D} = \frac{m_{interference}}{m_{diffraction}}$.

# Limits of Resolution and Circular Apertures

Lenses, acting as circular apertures of diameter *D*, cannot form perfect point images due to diffraction. The resolution of a lens, its ability to distinguish between two closely spaced objects, is limited by this effect. Light passing through a lens from a point source forms a diffraction pattern consisting of a central bright circular spot called the Airy disk, surrounded by fainter rings. The angular half-width *θ* of the Airy disk is approximately:

$$\theta \approx 1.22 \frac{\lambda}{D}$$

The **Rayleigh criterion** states that two point objects are just resolvable when the center of the diffraction pattern of one image aligns with the first minimum of the diffraction pattern of the other. The minimum angular separation *θ<sub>min</sub>* between two just-resolvable objects is:

$$\theta_{min} = 1.22 \frac{\lambda}{D}$$

This limit also applies to telescopes and mirrors, where *D* is the diameter of the objective. A smaller *θ<sub>min</sub>* indicates better resolution.

A **diffraction grating** consists of a large number of equally spaced parallel slits (separation *d*). Diffraction gratings typically have a very large number of slits per unit length, often thousands of lines per centimeter or millimeter. It is used for precise wavelength measurements. This precise relationship between the angle of diffraction and the wavelength makes diffraction gratings invaluable tools in **spectroscopy**, where they are used to separate and analyze the different wavelengths present in a light source. Principal maxima occur at angles *θ* given by:

$$\sin\theta = \frac{m\lambda}{d}, \quad m = 0, \pm 1, \pm 2, ...$$

The resolution of the **human eye** is limited by factors like cone spacing and pupil diameter, with a best angular resolution around $5 \times 10^{-4}$ radians.

The resolution of **telescopes and microscopes** is also limited by diffraction. For telescopes, the angular resolution is given by the Rayleigh criterion: $\theta_{min} = 1.22 \frac{\lambda}{D}$. For ground-based telescopes, the theoretical resolution limit imposed by diffraction is often not reached due to distortions caused by **atmospheric turbulence** (the blurring effect you see when looking at stars). For microscopes, the resolving power *s*, the minimum separation between resolvable object points, is: The resolving power of a microscope can be further enhanced by using **immersion oil** between the objective lens and the specimen. Immersion oil has a higher refractive index than air, which allows the objective lens to collect more of the diffracted light, thereby improving resolution. The resolving power *s* is given by:

$$s = \frac{1.22 \lambda f}{D}$$

where *f* is the focal length of the objective lens.

The **ultimate limit of resolution** for any optical instrument is approximately half the wavelength of the radiation used: $RP \approx \frac{\lambda}{2}$.
